In [1]:
########################################
''' Imports '''

from sklearn.datasets import load_files       
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
import numpy as np
from glob import glob
import os, csv
import random
from random import shuffle
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from keras.applications.resnet50 import ResNet50
from keras.applications.resnet50 import preprocess_input, decode_predictions
from keras.preprocessing import image 
from IPython.display import display 
from PIL import Image
from PIL import ImageFile 

from tqdm import tqdm

from keras.layers import Input
from keras import layers
from keras.layers import Dense
from keras.layers import Activation
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import GlobalMaxPooling2D
from keras.layers import ZeroPadding2D
from keras.layers import AveragePooling2D
from keras.layers import GlobalAveragePooling2D
from keras.layers import BatchNormalization
from keras.models import Model
from keras.optimizers import SGD
import keras.backend as K
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import decode_predictions
from keras.applications.imagenet_utils import preprocess_input
from keras.engine.topology import get_source_inputs

import tensorflow as tf
from object_detection.utils import dataset_util

print("Import finished")

Using TensorFlow backend.


Import finished


In [2]:
########################################
''' Class '''

class Traffic_Sign:
    """ A Single Traffic sign model """
    
    def __init__(self, filePath, fileName, width, height, x1, y1, x2, y2, classTxt, classID):
        self.filePath = filePath
        self.fileName = fileName
        self.width = width
        self.height = height
        self.x1 = x1
        self.y1 = y1
        self.x2 = x2
        self.y2 = y2
        self.classTxt = classTxt
        self.classID = classID  
        

In [3]:
########################################
''' Print helper function '''

# pretty print Traffic Sign object
def ts_pretty_print(ts):
    print("File: {} has name {}, with classID: {}".format(ts.fileName, ts.classTxt, ts.classID))
    


In [4]:
'''Mac'''

TSName_txt_path = "/Users/kevinsu/Desktop/Udacity/machine-learning-master/projects/capstone/TSR/Dataset/reducedSetTS.txt"
label_map_output_path = "/Users/kevinsu/Desktop/Udacity/machine-learning-master/projects/capstone/TSR/obj_detection/data/TS_label_map.pbtxt"

# All standard pictures in this folder are with its class name and .png format e.g. A1A.png
standard_folder = "/Users/kevinsu/Desktop/Udacity/machine-learning-master/projects/capstone/TSR/Standards"
standard_img_extension = ".png"

# dataset path
dataset_directory = "/Users/kevinsu/Desktop/Udacity/machine-learning-master/projects/capstone/TSR/Dataset"
training_dataset_directory = os.path.join(dataset_directory, "Training")
testing_dataset_directory = os.path.join(dataset_directory, "Testing")

# Store all images in png under training/testing folder
png_training_dataset_dir = os.path.join(dataset_directory, 'png_training')
png_testing_dataset_dir = os.path.join(dataset_directory, 'png_testing')

# tfRecords output path
tfRecordsDir = "/Users/kevinsu/Desktop/Udacity/machine-learning-master/projects/capstone/TSR/obj_detection/tfRecords"

'''Windows'''
'''
TSName_txt_path = "C:/Users/ksu/Desktop/ML/TSR_capstone/MLND_Traffic_Sign_Recognition/Dataset/reducedSetTS.txt"
label_map_output_path = "C:/Users/ksu/Desktop/ML/TSR_capstone/MLND_Traffic_Sign_Recognition/obj_detection/data/TS_label_map.pbtxt"

# All standard pictures in this folder are with its class name and .png format e.g. A1A.png
standard_folder = "C:/Users/ksu/Desktop/ML/TSR_capstone/MLND_Traffic_Sign_Recognition/Standards/"
standard_img_extension = ".png"

# dataset path
dataset_directory = "C:/Users/ksu/Desktop/ML/TSR_capstone/MLND_Traffic_Sign_Recognition/Dataset"
training_dataset_directory = os.path.join(dataset_directory, "Training")
testing_dataset_directory = os.path.join(dataset_directory, "Testing")

# Store all images in png under training/testing folder
png_training_dataset_dir = os.path.join(dataset_directory, 'png_training')
png_testing_dataset_dir = os.path.join(dataset_directory, 'png_testing')

# tfRecords output path
tfRecordsDir = "C:/Users/ksu/Desktop/ML/TSR_capstone/MLND_Traffic_Sign_Recognition/obj_detection/tfRecords"
'''


'\nTSName_txt_path = "C:/Users/ksu/Desktop/ML/TSR_capstone/MLND_Traffic_Sign_Recognition/Dataset/reducedSetTS.txt"\nlabel_map_output_path = "C:/Users/ksu/Desktop/ML/TSR_capstone/MLND_Traffic_Sign_Recognition/obj_detection/data/TS_label_map.pbtxt"\n\n# All standard pictures in this folder are with its class name and .png format e.g. A1A.png\nstandard_folder = "C:/Users/ksu/Desktop/ML/TSR_capstone/MLND_Traffic_Sign_Recognition/Standards/"\nstandard_img_extension = ".png"\n\n# dataset path\ndataset_directory = "C:/Users/ksu/Desktop/ML/TSR_capstone/MLND_Traffic_Sign_Recognition/Dataset"\ntraining_dataset_directory = os.path.join(dataset_directory, "Training")\ntesting_dataset_directory = os.path.join(dataset_directory, "Testing")\n\n# Store all images in png under training/testing folder\npng_training_dataset_dir = os.path.join(dataset_directory, \'png_training\')\npng_testing_dataset_dir = os.path.join(dataset_directory, \'png_testing\')\n\n# tfRecords output path\ntfRecordsDir = "C:/User

In [32]:
''' TS_label_map generator '''

def label_generator(TSName_62_collection):
    label_map_content = ""
    num_of_classes = 62
    for i in range(num_of_classes):
        label_map_content += "item{\n"
        label_map_content += "  name: \"{}\"\n".format(TSName_62_collection[i])
        label_map_content += "  id: {}\n".format(str(i+1))
        label_map_content += "}\n\n"
        
    with open(label_map_output_path, "w") as fout:
        fout.write(label_map_content) 

        
# create classID and className pairs for 62 Traffic Signs
with open(TSName_txt_path) as f:
    TSName_62 = f.readlines()
# remove whitespace characters
TSName_62 = [x.strip() for x in TSName_62[0:]]


# Generate label map for obj_detection
label_generator(TSName_62)

In [33]:
''' ppm to png converter generator '''

# png_dataset_dir: destination of png files of either training datasets and testing datasets
# img_path: the path to one single ppm image file. 

def convert_ppm_to_png(png_dataset_dir, ppm_img_path, ppm_img_name):
    # if this path doesn't exist, create 
    if not os.path.exists(png_dataset_dir):
        os.makedirs(png_dataset_dir)

    if ppm_img_name[-3:] == "ppm":
        img = Image.open(ppm_img_path)
        png_path = os.path.join(png_dataset_dir, ppm_img_name[:-3]+'png')
        #print(png_path)
        img.save(png_path)
        #img.show()
        return png_path
    else:
        print("Non ppm file detected: {}".format(ppm_img_name))
        

In [34]:
########################################
''' Load dataset into variable '''
    

# function to load training, test, and validation datasets into variables 
def load_dataset(path, png_dataset_dir):
    all_files = []
    print("Loading {}".format(path))
    for eachfolder in os.listdir(path):
        abs_loc = os.path.join(path, eachfolder)
        png_class_dir = os.path.join(png_dataset_dir, eachfolder)
        if not os.path.isdir(abs_loc):
            continue

        #open csv for this folder
        with open(abs_loc + "/GT-" + eachfolder + ".csv", "r") as csv_file:
            csv_reader = csv.reader(csv_file, delimiter=';')
            next(csv_reader, None) # skip header
            
            #create traffic sign object
            for row in csv_reader:
                abs_file_loc = os.path.join(abs_loc, row[0])
                png_file_loc = convert_ppm_to_png(png_class_dir, abs_file_loc, row[0]) #dir of png files, path of ppm file, name of ppm file
                all_files.append(Traffic_Sign(png_file_loc, row[0], row[1], row[2], row[3], row[4], row[5], row[6], TSName_62[int(row[7])], row[7]))
                
            csv_file.close()
        
        print("folder: {} completed".format(eachfolder))
    return all_files



# load train and test datasets into list of objects of Traffic signs
#all_training_objs = load_dataset("/Users/kevinsu/Desktop/Udacity/machine-learning-master/projects/capstone/TSR/Dataset/Training") # a list of training traffic_sign_collection
#all_testing_and_validation_objs = load_dataset("/Users/kevinsu/Desktop/Udacity/machine-learning-master/projects/capstone/TSR/Dataset/Testing")  # a list of testing traffic_sign_collection

all_training_objs = load_dataset(training_dataset_directory, png_training_dataset_dir) # a list of training traffic_sign_collection
all_testing_and_validation_objs = load_dataset(testing_dataset_directory, png_testing_dataset_dir)  # a list of testing traffic_sign_collection

# shuffle training objects and testing objects 
shuffle(all_training_objs)
shuffle(all_testing_and_validation_objs)

Loading /Users/kevinsu/Desktop/Udacity/machine-learning-master/projects/capstone/TSR/Dataset/Training
folder: 00031 completed
folder: 00009 completed
folder: 00036 completed
folder: 00000 completed
folder: 00038 completed
folder: 00007 completed
folder: 00053 completed
folder: 00054 completed
folder: 00039 completed
folder: 00006 completed
folder: 00001 completed
folder: 00008 completed
folder: 00037 completed
folder: 00030 completed
folder: 00055 completed
folder: 00052 completed
folder: 00048 completed
folder: 00041 completed
folder: 00046 completed
folder: 00012 completed
folder: 00015 completed
folder: 00023 completed
folder: 00024 completed
folder: 00047 completed
folder: 00040 completed
folder: 00049 completed
folder: 00025 completed
folder: 00022 completed
folder: 00014 completed
folder: 00013 completed
folder: 00057 completed
folder: 00050 completed
folder: 00059 completed
folder: 00061 completed
folder: 00035 completed
folder: 00032 completed
folder: 00004 completed
folder: 00

In [35]:
# random test of object
ts_pretty_print(all_training_objs[random.randint(0, len(all_training_objs))])
print("Total number of training obj: {}".format(str(len(all_training_objs))))
print("Total number of testing obj: {}".format(str(len(all_testing_and_validation_objs))))

File: 00035_00000.ppm has name E9a, with classID: 45
Total number of training obj: 4591
Total number of testing obj: 2534


In [12]:
def create_TS_tf_example(TS_obj): 
    height = int(TS_obj.height)
    width = int(TS_obj.width)
    filename = TS_obj.fileName
    
    with tf.gfile.GFile(TS_obj.filePath, 'rb') as fid:
        encoded_image_data = fid.read()

    image_format = b'png'

    xmins = [float(TS_obj.x1) / float(width)]
    xmaxs = [float(TS_obj.x2) / float(width)]
    ymins = [float(TS_obj.y1) / float(height)]
    ymaxs = [float(TS_obj.y2) / float(height)]
    
    classes_text = [TS_obj.classTxt.encode('utf8')]
    classes = [int(TS_obj.classID)]

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename.encode('utf8')),
        'image/source_id': dataset_util.bytes_feature(filename.encode('utf8')),
        'image/encoded': dataset_util.bytes_feature(encoded_image_data),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example

In [13]:
# define tfRecord writter 
tfRecordsPath = os.path.join(tfRecordsDir, "training.tfrecords")
writer = tf.python_io.TFRecordWriter(tfRecordsPath)

for TS_obj in all_training_objs:
    tf_example = create_TS_tf_example(TS_obj)
    writer.write(tf_example.SerializeToString())

writer.close()
print("Finish converting training objects to tfRecords")

Finish converting training objects to tfRecords


In [ ]:
''' Modify config files of the chosen pre-trained model '''
''' Chosen model: faster_rcnn_inception_v2_coco '''
# run command in terminal to train
# python3 train.py --logtostderr --train_dir=./models/train --pipeline_config_path=faster_rcnn_inception_v2_coco.config

# python3 export_inference_graph.py --input_type image_tensor --pipeline_config_path ./faster_rcnn_inception_v2_coco.config --trained_checkpoint_prefix ./models/train/model.ckpt-20000 --output_directory ./fine_tuned_model

In [ ]:
class TS_detector:
    def __init__(self):
        frozen_model_path = '/Users/kevinsu/Desktop/Udacity/machine-learning-master/projects/capstone/TSR/obj_detection/fine_tuned_model/frozen_inference_graph.pb'
        self.detection_graph = tf.Graph()
        with self.detection_graph.as_default():
            od_graph_def = tf.GraphDef()

            with tf.gfile.GFile(frozen_model_path, 'rb') as fid:
                serialized_graph = fid.read()
                od_graph_def.ParseFromString(serialized_graph)
                tf.import_graph_def(od_graph_def, name='')
            self.image_tensor = self.detection_graph.get_tensor_by_name('image_tensor:0')
            self.d_boxes = self.detection_graph.get_tensor_by_name('detection_boxes:0')
            self.d_scores = self.detection_graph.get_tensor_by_name('detection_scores:0')
            self.d_classes = self.detection_graph.get_tensor_by_name('detection_classes:0')
            self.num_d = self.detection_graph.get_tensor_by_name('num_detections:0')
        self.sess = tf.Session(graph=self.detection_graph)
    
    def get_classification(self, img):
        # Bounding Box Detection.
        with self.detection_graph.as_default():
            # Expand dimension since the model expects image to have shape [1, None, None, 3].
            img_expanded = np.expand_dims(img, axis=0)    
            (boxes, scores, classes, num) = self.sess.run(
                [self.d_boxes, self.d_scores, self.d_classes, self.num_d],
                feed_dict={self.image_tensor: img_expanded})
        return boxes, scores, classes, num

my_detector = TS_detector()

def obj_detection_predict (img_dir, correctID):
    # my_img_path = "/Users/kevinsu/Desktop/Udacity/machine-learning-master/projects/capstone/TSR/Dataset/png_testing/00019/00407_00001.png"
    all_files = []
    correct_num = 0
    for file in os.listdir(img_dir):
        if file[-3:] == "png":
            all_files.append(os.path.join(img_dir, file))

    filename_queue = tf.train.string_input_producer(all_files) #  list of files to read

    reader = tf.WholeFileReader()
    key, value = reader.read(filename_queue)

    my_img = tf.image.decode_png(value) # use png or jpg decoder based on your files.
    
    init_op = tf.global_variables_initializer()
    with tf.Session() as sess:
        sess.run(init_op)

        # Start populating the filename queue.

        coord = tf.train.Coordinator()
        threads = tf.train.start_queue_runners(coord=coord)

        for i in range(len(all_files)): #length of your filename list
            image = my_img.eval() #here is your image Tensor :) 
            boxes, scores, classes, num = my_detector.get_classification(image)
            #print(image.shape)
            #Image.fromarray(np.asarray(image)).show()
            boxes, scores, classes, num = my_detector.get_classification(image)
            '''
            print("boxes")
            print(boxes[0][0])
            print("scores")
            print(scores[0][0])
            print("classes")
            print(classes[0][0])
            print("num")
            print(num)
            '''
            if classes[0][0] == correctID:
                correct_num += 1

        coord.request_stop()
        coord.join(threads)
        print("Class {}: Accuracy: {}. {} correct predictions in {} pictures".format(correctID, float(correct_num)/len(all_files), correct_num, len(all_files)))
        return correct_num
        
test_dir = "/Users/kevinsu/Desktop/Udacity/machine-learning-master/projects/capstone/TSR/Dataset/png_testing"
list_of_classID = []
list_of_correct_predictions = []
for dir in os.listdir(test_dir):
    if len(dir) == 5:
        list_of_classID.append(dir[-2:])
        cor_pred = obj_detection_predict(os.path.join(test_dir, dir), float(dir[-2:]))
        list_of_correct_predictions.append(cor_pred)


Class 31.0: Accuracy: 0.8953488372093024. 77 correct predictions in 86 pictures
Class 0.0: Accuracy: 0.0. 0 correct predictions in 6 pictures
Class 38.0: Accuracy: 0.9436619718309859. 201 correct predictions in 213 pictures
Class 7.0: Accuracy: 0.03333333333333333. 3 correct predictions in 90 pictures
Class 53.0: Accuracy: 0.9583333333333334. 23 correct predictions in 24 pictures
Class 54.0: Accuracy: 1.0. 48 correct predictions in 48 pictures


In [52]:
print(list_of_classID)
for dir in os.listdir(test_dir):
        print(dir)
print("Accuracy score: {}".format(100*np.sum(list_of_correct_predictions)/len(all_testing_and_validation_objs)))

['31', '00', '38', '07', '53', '54', '39', '06', '01', '08', '37', '30', '55', 're']
00031
00000
00038
00007
00053
00054
00039
00006
00001
00008
00037
00030
00055
.DS_Store
00041
00046
00012
00023
00024
00047
00040
00049
00025
00022
00014
00013
00057
00059
00061
00035
00032
00004
00003
00060
00058
00051
00056
00002
00005
00034
00016
00029
00027
00018
00020
00045
00042
00021
00019
00010
00017
00028
00043
00044
Accuracy score: 20.047355958958168
